In [1]:
# Imports & read file
import time
import itertools
import re

def read_file(filename):
    with open(filename) as infile:
        return [list(g) for k, g in itertools.groupby([line.strip() for line in infile.readlines()], lambda x: bool(x)) if k]
    return None

In [2]:
# Part 1
def rotations(tile):
    yield tile
    for _ in range(3):
        tile = [''.join(tile[len(tile)-1-c][r] for c in range(len(tile))) for r in range(len(tile))]
        yield tile
        
def flip(tile):
    yield tile
    yield list(reversed(tile))
    
def orientations(tile):
    return [f for r in rotations(tile) for f in flip(r)]

def all_orientations(tiles):
    return {int(tile[0][5:-1]): orientations(tile[1:]) for tile in tiles}

def top(tile):
    return tile[0]

def bottom(tile):
    return tile[-1]

def left(tile):
    return ''.join(t[0] for t in tile)

def right(tile):
    return ''.join(t[-1] for t in tile)

def mult_corners(tiles):
    sides = {}
    for t in tiles:
        for o in tiles[t]:
            sides[top(o)] = sides.get(top(o), []) + [t]
    uniques = {s: sides[s][0] for s in sides if len(sides[s]) == 1}
    tiles_wu = {}
    for u in uniques:
        tiles_wu[uniques[u]] = tiles_wu.get(uniques[u], 0) + 1
    A, B, C, D = [t for t in tiles_wu if tiles_wu[t] == 4]
    return A * B * C * D

In [3]:
# Test part 1
start = time.time()
print(mult_corners(all_orientations(read_file("test01.txt"))) == 20899048083289)
time.time() - start

True


0.0009999275207519531

In [4]:
# Solve part 1
start = time.time()
print(mult_corners(all_orientations(read_file("input.txt"))))
time.time() - start

14129524957217


0.022996902465820312

In [5]:
# Part 2
def compose(tiles):
    # Sort tiles into corner/edge/middle pieces
    sides = {}
    for t in tiles:
        for o in tiles[t]:
            sides[top(o)] = sides.get(top(o), []) + [t]
    nbs = {t: set() for t in tiles}
    for share in sides.values():
        if len(share) == 2:
            nbs[share[0]].add(share[1])
            nbs[share[1]].add(share[0])
    corners = set(t for t in nbs if len(nbs[t]) == 2)
    edges = set(t for t in nbs if len(nbs[t]) == 3)
    middle = set(t for t in nbs if len(nbs[t]) == 4)
    levels = {0: corners, 1: edges, 2: middle}
    
    # Which tile goes where?
    size = int(len(tiles)**0.5)
    picture = [[None for _ in range(size)] for _ in range(size)]
    used = set()
    for row in range(size):
        for col in range(size):
            level = 2
            if row == 0 or row == size - 1:
                level -= 1
            if col == 0 or col == size - 1:
                level -= 1
            match = set()
            if row > 0:
                match.add(picture[row - 1][col])
            if col > 0:
                match.add(picture[row][col - 1])
            for tile in levels[level] - used:
                if len(nbs[tile] & match) == len(match):
                    picture[row][col] = tile
                    used.add(tile)
                    break
    
    # Rotate tiles to fit appropriately
    outside = {side for side in sides if len(sides[side]) == 1}
    for row in range(size):
        for col in range(size):
            N = outside if row == 0 else {bottom(picture[row - 1][col])}
            S = outside if row == size - 1 else {side for side in sides if picture[row][col] in sides[side] and picture[row+1][col] in sides[side]}
            W = outside if col == 0 else {right(picture[row][col - 1])}
            E = outside if col == size - 1 else {side for side in sides if picture[row][col] in sides[side] and picture[row][col+1] in sides[side]}
            for tile in tiles[picture[row][col]]:
                if top(tile) in N and bottom(tile) in S and left(tile) in W and right(tile) in E:
                    picture[row][col] = tile
                    break
    return ["".join(picture[row][col][line][1:-1] for col in range(size)) for row in range(size) for line in range(1,9)]

def find_snek(picture):
    snek = re.compile(r"(?=([^|]{{18}}#[^|].{{{0}}}#[^|]{{4}}##[^|]{{4}}##[^|]{{4}}###.{{{0}}}[^|]#[^|]{{2}}#[^|]{{2}}#[^|]{{2}}#[^|]{{2}}#[^|]{{2}}#[^|]{{3}}))".format(len(picture)-19))
    sea = "|".join(picture)
    return len(snek.findall(sea))

def roughness(picture):
    c = ''.join(picture).count('#')
    return c - 15 * max(find_snek(o) for o in orientations(picture))

In [6]:
# Test part 2
start = time.time()
print(roughness(compose(all_orientations(read_file("test01.txt")))) == 273)
time.time() - start

True


0.0019969940185546875

In [7]:
# Solve part 2
start = time.time()
print(roughness(compose(all_orientations(read_file("input.txt")))))
time.time() - start

1649


0.05098772048950195